In [1]:
import pandas as pd
import json

In [2]:
FILE = "AMAZON_FASHION_5.json"

In [3]:
df = pd.DataFrame()

In [62]:
with open(FILE, 'r') as f:
    for json_line in f.readlines():
        line = json.loads(json_line)
        if line.get("style"):
            for key, value in line.pop("style").items():
                line[f"style.{key}"] = value
        df = df.append(line, ignore_index=True)

In [63]:
reviews_by_user = df.groupby(by="reviewerID").count()

In [64]:
reviews_by_user.max()

overall             16
verified            16
reviewTime          16
asin                16
reviewerName        16
reviewText          16
summary             16
unixReviewTime      16
style.Size:         16
style.Color:        16
style.Size Name:     1
style.Style:         1
vote                16
image               16
dtype: int64

In [65]:
reviews_by_user.min()

overall             4
verified            4
reviewTime          4
asin                4
reviewerName        4
reviewText          0
summary             4
unixReviewTime      4
style.Size:         0
style.Color:        0
style.Size Name:    0
style.Style:        0
vote                0
image               0
dtype: int64

In [66]:
df.to_json("AMAZON_FASION_5_formatted.json")

In [41]:
df = pd.read_json("AMAZON_FASION_5_formatted.json")

In [42]:
df = df[["reviewerID", "overall", "asin"]]

In [43]:
df.drop_duplicates(inplace=True)

In [44]:
df.reset_index(inplace=True)

In [45]:
df = df.pivot_table(index='reviewerID', columns='asin', values='overall')

# Changing the sparsity of the matrix

This could be done either by looping of each row until the desired amount of dropped cells is reached (implemented) or we could choose rows at random and drop cells untill the desired amount is reached. This way not only the first X rows will drop more than the others, but it would be divided

In [57]:
import numpy as np

In [58]:
sparsity = lambda dataf: dataf.isna().sum().sum() / (dataf.shape[0] * dataf.shape[1])

In [59]:
total = df.shape[0] * df.shape[1]

In [60]:
sparsity(df)

0.8047830923248054

In [61]:
desired_percentage = .8

In [62]:
cells_to_drop = (desired_percentage * total) - current_nan

In [64]:
cells_dropped = 0
while True:
    for index, row in df.iterrows():
        nans = np.where(row.notnull())[0]
        if not len(nans):
            continue
        drop = np.random.choice(nans)
        row[drop] = np.nan
        cells_dropped += 1
        if cells_dropped >= cells_to_drop:
            break    
    if cells_dropped >= cells_to_drop:
        break
        


In [65]:
cells_dropped

525

In [66]:
sparsity(df)

0.8464961067853171